In [80]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

### read all data

In [68]:
!pwd

/Users/dengwenning/Documents/GitHub/OneHourOneLife/2_demographics


In [70]:
rift_data = pd.read_csv('../data/outputs/lifelogs_bigserver2_rift_data.tsv', sep='\t')
arc_data = pd.read_csv('../data/outputs/lifelogs_bigserver2_arc_data.tsv', sep='\t')
boundless_data = pd.read_csv('../data/outputs/lifelogs_bigserver2_boundless_data.tsv', sep='\t')
rift_data.head()

,Unnamed: 0,server,release,era,event,timestamp,playerID,hash,age,sex,location,parent,cause_of_death,pop,chain,killer
0,0,bigserver2,253.0,rift,D,1564099205,1806575.0,91f6003fa6a03933f72b1f95a7002f1f07928deb,33.40,M,"(277, -2434)",NaN,hunger,67,NaN,NaN
1,1,bigserver2,253.0,rift,D,1564099220,1806488.0,1a7f24be09f45519e96d4f017b36ad38eb5e329d,60.00,M,"(-2516, -1279)",NaN,oldAge,66,NaN,NaN
2,2,bigserver2,253.0,rift,D,1564099234,1806629.0,3dc579b91219e519dfee2f9d7a24aec8e9a83b03,19.04,F,"(-1919, -2078)",NaN,hunger,65,NaN,NaN
3,3,bigserver2,253.0,rift,D,1564099254,1806530.0,780359de9275c29b15b65d3cdba5b22bf90169c5,47.75,F,"(-2525, -1276)",NaN,hunger,64,NaN,NaN
4,4,bigserver2,253.0,rift,D,1564099256,1806662.0,bcc9ac263a4036f0141ffb04e9224e00443bb791,10.92,F,"(277, -2433)",NaN,hunger,64,NaN,NaN


### Specify which data we are looing at

In [97]:
lifelog_data = boundless_data

get death date

In [98]:

death_data = lifelog_data[lifelog_data['event'] == 'D'].reset_index(drop=True)
death_data.head()

,Unnamed: 0,server,release,era,event,timestamp,playerID,hash,age,sex,location,parent,cause_of_death,pop,chain,killer
0,1,bigserver2,282.0,boundless,D,1573257621,2251778.0,c17d2f3663837c82e4f7669a40aa6f618e89e57a,0.54,F,"(-361, -410)",NaN,hunger,78,NaN,NaN
1,4,bigserver2,282.0,boundless,D,1573257624,2251740.0,bd6841eb13118ce2a0453e5802b91f7ec0c1e20e,3.72,F,"(-238, -855)",NaN,hunger,80,NaN,NaN
2,5,bigserver2,282.0,boundless,D,1573257625,2251577.0,7e058600565826811059c93c9ce403d7b6c4533b,21.72,M,"(-201, -903)",NaN,hunger,80,NaN,NaN
3,6,bigserver2,282.0,boundless,D,1573257626,2251776.0,40a960756347fa0a7c3d14791bd0379c6cc754cf,0.69,M,"(-765, -581)",NaN,hunger,79,NaN,NaN
4,8,bigserver2,282.0,boundless,D,1573257632,2251722.0,ac7e03fde959b54812abf9798cc25cf55b3d6baa,5.90,M,"(-323, -508)",NaN,hunger,77,NaN,NaN


get unique player hash

In [99]:
big_hash = death_data.groupby('hash')['timestamp'].count().reset_index()
big_hash = big_hash.rename(columns={"hash": "player", "timestamp": "n_lives"})
big_hash.head()

,player,n_lives
0,0002e5ea5ce7cfd761135d255a245a3344af4377,338
1,00047b60a01d908e2407b05a09b404c2f6cec6f6,4
2,0004f2a9005cf3562482fe408876808498e1fad5,11
3,000548ed7f04e2ddf81b4016d87d53ddbbd2e89f,45
4,0005eb266ff93d00bb6072d9c07f3a47d4864219,1


### create a new data frame that incorporates cumulative and total number of lives 

In [100]:
death_data_new = death_data[['release','timestamp','hash','playerID','age','cause_of_death']].copy()
death_data_new['n_lives'] = np.inf
death_data_new['cumulative_game_time'] = np.inf

In [101]:
death_data_new.head()

,release,timestamp,hash,playerID,age,cause_of_death,n_lives,cumulative_game_time
0,282.0,1573257621,c17d2f3663837c82e4f7669a40aa6f618e89e57a,2251778.0,0.54,hunger,inf,inf
1,282.0,1573257624,bd6841eb13118ce2a0453e5802b91f7ec0c1e20e,2251740.0,3.72,hunger,inf,inf
2,282.0,1573257625,7e058600565826811059c93c9ce403d7b6c4533b,2251577.0,21.72,hunger,inf,inf
3,282.0,1573257626,40a960756347fa0a7c3d14791bd0379c6cc754cf,2251776.0,0.69,hunger,inf,inf
4,282.0,1573257632,ac7e03fde959b54812abf9798cc25cf55b3d6baa,2251722.0,5.90,hunger,inf,inf


In [ ]:
newdf = pd.DataFrame(columns = ['release','timestamp','hash','playerID','age','cause_of_death','n_lives','cumulative_game_time'])
for i in tqdm(death_data.hash.unique()):
    sub_data = death_data_new.query('hash == @i').copy().reset_index(drop = True)
    sub_data['cumulative_game_time'] = sub_data.index +1
    sub_data['n_lives'] = big_hash.query('player == @i')['n_lives'].tolist()[0]
    newdf = newdf.append(sub_data, sort = False)
      

A Jupyter Widget

In [103]:
newdf = newdf.sort_values(by = ['release','timestamp']).reset_index(drop = True)

export data

In [104]:
newdf.to_csv('../data/outputs/expertise_boundless_data.csv',index=False)